In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [2]:
def import_data(i):
    
    country_index = pd.read_csv("country_index.csv")
    pred_country_index = pd.read_csv("submission_country_index.csv")
    
    first_index = country_index.loc[i][1]
    last_index = country_index.loc[i][2]
    tile_value = country_index.loc[i][3]
    code = country_index.loc[i][0]
    first_index_sub = pred_country_index.loc[i][1]
    last_index_sub = pred_country_index.loc[i][2]
    print(code)
    df1 = pd.read_csv("gee_features.csv", 
                       skiprows=range(1, 1+first_index), 
                       nrows=(last_index - first_index + 1))
           
    return df1

In [3]:
def filter_data(df1):
    
    df1 = df1.drop(['new_ind', 'index', 'ADM1DHS', 'ADM1FIPS', 'ADM1FIPSNA', 'ADM1NAME', 'ADM1SALBCO', 'ADM1SALBNA', 'ADM1SALNA', 'ALT_DEM', 'ALT_GPS', 'CCFIPS', 'DATUM', 'DHSCC', 'DHSCLUST', 'DHSREGCO', 'DHSREGNA', 'DHSYEAR', 'F21', 'F22', 'F23', 'LATNUM', 'LONGNUM', 'SOURCE', 'URBAN_RURA', 'ZONECO', 'ZONENA','key1','key2','key3'], axis = 1)
    df1 = df1.dropna(axis=1)
    df1 = df1.sort_index(axis=1)
    dhsid = np.array(df1["DHSID"])

    x12 = list(df1.columns)
    x12.remove("DHSID")
    
    df_info = {
    'name' : x12,
    'len' : [],
    'element' : [],
    'min' : [],
    'max' : [],
    'top_value_count' : [],
    }
    for i in x12:
        w1 = list(set(df1[i]))
        df_info['max'].append(max(w1))
        df_info['min'].append(min(w1))
        df_info['element'].append(len(w1))
        df_info['len'].append(len(list(df1[i])))
        df_info['top_value_count'].append(df1[i].value_counts().values[0])

    df_info = pd.DataFrame(df_info) 
    
    
    qaw = list(df_info[(df_info['max'] == 65535.000000) & (df_info['element'] > 60)]['name'])
    for i in qaw:
        sq = np.array(list(df1[i]))
        sq[sq == 65535.000000] = np.mean(sq[sq != 65535.000000])
        df1[i] = sq
        
    qaw = list(df_info[(df_info['max'] == 255.000000) & (df_info['element'] > 60)]['name'])
    for i in qaw:
        sq = np.array(list(df1[i]))
        sq[sq == 255.000000] = np.mean(sq[sq != 255.000000])
        df1[i] = sq

    df_info = {
    'name' : x12,
    'len' : [],
    'element' : [],
    'min' : [],
    'max' : [],
    'top_value_count' : [],
    }
    for i in x12:
        w1 = list(set(df1[i]))
        df_info['max'].append(max(w1))
        df_info['min'].append(min(w1))
        df_info['element'].append(len(w1))
        df_info['len'].append(len(list(df1[i])))
        df_info['top_value_count'].append(df1[i].value_counts().values[0])

    df_info = pd.DataFrame(df_info)
    
    cat_1 = list(df_info[(df_info['element'] >= 775)]['name'])
    cat_2 = list(df_info[(df_info['element'] < 60) & (df_info['element'] != 1)]['name'])
    cat_3 = list(df_info[(df_info['element'] < 775) & (df_info['element'] >= 60)]['name'])
    
    return cat_1, cat_2, cat_3, df1, dhsid

In [4]:
def updatearray_reg(X,deg,check):
    yset = list(set(X))
    yset.sort()
    X = np.array(X).reshape(-1)
    xset = np.arange(len(yset)).reshape(-1,1)
    poly = PolynomialFeatures(degree=deg)
    x_poly = poly.fit_transform(xset)
    model = LinearRegression()
    model.fit(x_poly, yset)
    y_pred = model.predict(x_poly)
    for i in range(len(yset)):
        X[X == yset[i]] = y_pred[i]

    if check > 0:
        mean = np.mean(X)
        std = np.std(X)
        X = (X - mean) / std
        X = X*check
    return X 

In [ ]:
def unique_cat(arr):
    unique_values = np.unique(arr)
    unique_values.sort()
    converted_arr = np.searchsorted(unique_values, arr)

    return converted_arr

In [ ]:
def final_data(cat_1, cat_2, cat_3, df1):
    df_cat1 = {}
    for i in cat_1:
        df_cat1[i]= updatearray_reg(list(df1[i]),2,10)
    df_cat1 = pd.DataFrame(df_cat1)
    
    df_cat2 = {}
    for i in cat_2:
        df_cat2[i] = unique_cat(list(df1[i]))
    for i in cat_3:
        df_cat2[i] = updatearray_reg(list(df1[i]),3,5)
    df_cat2 = pd.DataFrame(df_cat2)
    
    df_final = pd.concat([df_cat1, df_cat2], axis=1)
    
    print(df_final.shape)
    
    return df_final

In [ ]:
def show_plot_pca(df_final):
    data = df_final.values
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    pca = PCA()
    pca.fit(data)
    explained_variance_ratio = pca.explained_variance_ratio_
    cumulative_variance_ratio = np.cumsum(explained_variance_ratio)
    num_components = len(explained_variance_ratio)
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, num_components + 1), cumulative_variance_ratio, marker='o', linestyle='-')
    plt.xlabel('Number of Components')
    plt.ylabel('Cumulative Explained Variance Ratio')
    plt.title('Cumulative Explained Variance Ratio vs. Number of Components')
    plt.grid()
    plt.show()
    

In [ ]:
def apply_pca_with_standardization(X, j, n_components, dhsid, apply_standardization=False):
    back = len(X)
    X = np.tile(X, (j, 1))
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    if apply_standardization:
        scaler = StandardScaler()
        X_pca = scaler.fit_transform(X_pca)
        
    transformed_df_final = pd.DataFrame(np.array(X_pca[:back]))
        
    transformed_df_final['DHSID'] = dhsid
    df_train = pd.read_csv("training_label.csv")
    df_sub = pd.read_csv("sample submission (1).csv")
    merged_df = pd.merge(df_train, transformed_df_final, on='DHSID')
    merged_df_sub = pd.merge(df_sub, transformed_df_final, on='DHSID')
    dhsid_sub = np.array(merged_df_sub["DHSID"])
    
    df_created_r2 = {
    'feature' : list(range(0,n_components)),
    'Mean_BMI' : [merged_df[i].corr(merged_df['Mean_BMI']) for i in range(n_components)] ,
    'Median_BMI' : [merged_df[i].corr(merged_df['Median_BMI']) for i in range(n_components)],
    'Unmet_Need_Rate' : [merged_df[i].corr(merged_df['Unmet_Need_Rate']) for i in range(n_components)],
    'Under5_Mortality_Rate' : [merged_df[i].corr(merged_df['Under5_Mortality_Rate']) for i in range(n_components)],
    'Skilled_Birth_Attendant_Rate' : [merged_df[i].corr(merged_df['Skilled_Birth_Attendant_Rate']) for i in range(n_components)],
    'Stunted_Rate' : [merged_df[i].corr(merged_df['Stunted_Rate']) for i in range(n_components)],
    }
    df_created_r2 = pd.DataFrame(df_created_r2)
        
    return merged_df, merged_df_sub, df_created_r2, dhsid_sub

In [ ]:
def train_data(merged_df, merged_df_sub, df_created_r2, name):
    
    feature_map = list(df_created_r2[(df_created_r2[name] < -0.03) | (df_created_r2[name] > 0.03)]['feature'])
    
    f2 = feature_map.copy()
    f2.append(name)
    sd1 = merged_df[f2]
    x_sub = merged_df_sub[feature_map]
    
    sd2 = sd1.dropna(axis=0)
    
    X_value = sd2[feature_map]
    Y_value = sd2[name]
    
    x_train, x_test, y_train, y_test = train_test_split(X_value, Y_value, test_size=0.2, random_state=42)
    #------------------------------------------------------
    from sklearn.linear_model import ElasticNet
    ElasticNet = ElasticNet()
    param_grid1 = {
        'alpha': [0.1, 0.15, 0.2, 0.25,0.3,0.35, 0.4,0.45, 0.5,0.55, 0.6,0.65, 0.7,0.75,0.8,0.85, 0.9,1],
        'l1_ratio': [0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    }
    grid_search1 = GridSearchCV(estimator=ElasticNet, param_grid=param_grid1, scoring='r2', cv=5)
    grid_search1.fit(x_train, y_train)
    best_model1 = grid_search1.best_estimator_
    best_params1 = grid_search1.best_params_
    elastic_net_preds = best_model1.predict(x_test)
    accuracy1 = np.sqrt(mean_squared_error(y_test, elastic_net_preds))
    
    #------------------------------------------------------
    linear_reg = LinearRegression()
    param_grid2 = {
    'fit_intercept': [True, False],
    'normalize': [True, False]
    }
    grid_search2 = GridSearchCV(estimator=linear_reg, param_grid=param_grid2, scoring='r2', cv=5)
    grid_search2.fit(x_train, y_train)
    best_model2 = grid_search2.best_estimator_
    best_params2 = grid_search2.best_params_
    linear_reg_preds = best_model2.predict(x_test)
    accuracy2 = np.sqrt(mean_squared_error(y_test, linear_reg_preds))
    
    #------------------------------------------------------
    values = [accuracy1, accuracy2]
    minimum_value = min(values)
    p = values.index(minimum_value)
    minx = min(list(Y_value))
    manx = max(list(Y_value))
    
    if p == 0:
        y_pred_sub = best_model1.predict(x_sub)
        info = accuracy1    
        return info, y_pred_sub, minx, manx
    if p == 1:
        y_pred_sub = best_model2.predict(x_sub)
        info = accuracy2
        return info, y_pred_sub, minx, manx

In [ ]:
df1= import_data(50)
cat_1, cat_2, cat_3, df1, dhsid = filter_data(df1)
df_final = final_data(cat_1, cat_2, cat_3, df1)
show_plot_pca(df_final)

In [ ]:
merged_df, merged_df_sub, df_created_r2, dhsid_sub = apply_pca_with_standardization(df_final, 2, 1000, dhsid, apply_standardization=True)

In [ ]:
names = ['Mean_BMI','Median_BMI','Unmet_Need_Rate','Under5_Mortality_Rate','Skilled_Birth_Attendant_Rate', 'Stunted_Rate']

y_p = []
rmse = []
max_value1 = []
y_max_min = []
for name in names:
    info, y_pred_sub, minx, manx = train_data(merged_df, merged_df_sub, df_created_r2, name)
    y_pred_sub = list(y_pred_sub)
    y_abs = [abs(num) for num in y_pred_sub]
    max_value = max(y_abs)
    max_value1.append(max_value)
    y_p.append(y_pred_sub)
    rmse.append(info)
    y_max_min.append([minx,manx])
    
dic = {
     "dhsid": list(dhsid_sub),
     "mean": y_p[0],
     "median": y_p[1],
     "unmet": y_p[2],
     "under5": y_p[3],
     "skilled": y_p[4],
     "stunt": y_p[5]
 }   
                
dic = pd.DataFrame(dic)     





In [ ]:
rmse

In [ ]:
mrmse = np.mean(rmse)
mrmse

In [ ]:
dic.to_csv("new.csv", index = False)

In [ ]:
max_value1